# <span style="color:#F72585">Clasificación de imágenes con Pytorch-lightning</span>

<figure>
<center>
<img src="../Imagenes/trainer.png" width="800" height="800" align="center"/>
</center>
</figure>


Fuente: Alvaro Montenegro

## <span style="color:blue">Introducción</span> 

En esta lección aprendemos como construir una modelo de clasificación de imágenes a color. Usaremos el framework [Pytorch-lightning](https://www.pytorchlightning.ai/) para construir el modelo y el conjunto de datos [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). La siguiente figura los tipos de datos que se usarán en esta lección. El contenido es una adaptación libre del tutorial [Image Classification using PyTorch Lightning](https://wandb.ai/wandb/wandb-lightning/reports/Image-Classification-using-PyTorch-Lightning--VmlldzoyODk1NzY) de [WandB](https://wandb.ai/site).

<figure>
<center>
<img src="../Imagenes/cifar10.png" width="600" height="600" align="center"/>
</center>
</figure>


Fuente: [Universidad de Toronto-Cifar10](https://www.cs.toronto.edu/~kriz/cifar.html)

Para esta lección usaremos los datos CIFAR10 disponibles en los datasets de la librería [Torchvision](https://pytorch.org/vision/stable/index.html). Adicionalmente usaremos [Weights and bias-Wandb](https://wandb.ai/site), una plataforma moderna que puede apoyar para crear mejores modelos, más rápido con el seguimiento de experimentos, el control de versiones de conjuntos de datos y la gestión de modelos. WandB debe instalarse por separado, pero se incorpora a la librería `pytorch_lightning.loggers`.


## <span style="color:blue">Carga las librerías requeridas</span> 

In [40]:
import torch
import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F

# Carga DataLoader para crear los dataloaders
from torch.utils.data import DataLoader, random_split

# Librería Torchvision
import torchvision
from torchvision import transforms

# métricas
import torchmetrics 

# reproductibilidad
from pytorch_lightning import seed_everything # reproducibilidad

# Carga WandBLogger para hacer seguimiento (tracking) del entrenamiento
from pytorch_lightning.loggers import WandbLogger

# carga el dataset CIFAR10
import torchvision.datasets as datasets
CIFAR10 = datasets.CIFAR10

# callbacks
from pytorch_lightning.callbacks import Callback 
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint


## <span style="color:blue">Preparación de los datos</span> 

Los `DataModule` son una forma de desacoplar enlaces relacionados con datos del LightningModule para que pueda desarrollar modelos agnósticos de conjuntos de datos. En otra palabras, con DataModule puede preparar los datos por fuera del módulo de entrenamiento, de tal manera que peude cambiar sus datos sin tocar el módulo de entrenamiento.

Con *DataModule* podemos organiza la canalización de datos en una clase compartible y reutilizable. Un módulo de datos encapsula los cinco pasos involucrados en el procesamiento de datos en PyTorch:

* Descargar/tokenizar/procesar.
* Limpiar y (tal vez) guardar en el disco.
* Cargar dentro del conjunto de datos.
* Aplicar transformaciones (rotar, tokenizar, etc.).
* Envolver dentro de un DataLoader.


Obtenga más información sobre los módulos de datos [aquí](https://pytorch-lightning.readthedocs.io/en/latest/data/datamodule.html). Construyamos un módulo de datos para el conjunto de datos Cifar-10.

### <span style="color:#4CC9F0">Canalización (pipeline) de los datos con DataModule</span>

Construimos una clase derivada de `LightningDataModule` específica para CIFAR10.

In [3]:
class CIFAR10DataModule(pl.LightningDataModule):
    
    def __init__(self, batch_size, data_dir: str = './', num_workers=4):
        """
        Pasaremos los hiperparámetros necesarios para nuestra canalización de datos
        utilizando el método __init__. También definiremos la canalización de 
        transformación de datos aquí.
        params:
        datadir
        """
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers


        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        self.dims = (3, 32, 32)
        self.num_classes = 10
        
        
    def prepare_data(self):
        """
        Aquí es donde definiremos la lógica para descargar nuestro conjunto de datos. 
        Estamos utilizando la clase de conjunto de datos CIFAR10 de torchvision para descargar.
        Use este método para hacer cosas que podrían escribirse en el disco 
        o que deben hacerse solo desde una única GPU en configuraciones distribuidas. 
        No haga ninguna asignación de estado en esta función 
        (es decir, self.alguna_cosa = ...).
        """
        # descarga
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        """
        Aquí es donde cargaremos los datos del archivo y prepararemos 
        los conjuntos de datos del tensor PyTorch para cada división de los datos. 
        La división de datos es por lo tanto reproducible. 
        Este método espera un argumento de etapa (stage) que se utiliza para separar 
        la lógica de 'entrenamiento' y de 'prueba'. 
        Esto es útil si no queremos cargar todo el conjunto de datos a la vez. 
        Las operaciones de datos que queremos realizar en cada GPU se definen aquí. 
        Esto incluye aplicar la transformación al conjunto de datos del tensor de PyTorch.
        """        
        # Asigna datos a los datasets de entrenamiento/validación  
        # para uso en los dataloaders
        if stage == 'fit' or stage is None:
            cifar_full = CIFAR10(self.data_dir, train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        # Asigna dataset de prueba para uso en los  dataloader(s)
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform)

    # Métodos para crear los dataloaders        
    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)


    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size, num_workers=self.num_workers)


    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size, num_workers=self.num_workers)


## <span style="color:blue">Devolución de llamadas (Callbacks)</span> 

Una devolución de llamada o `callback` es un programa autónomo que se puede reutilizar en todos los proyectos. PyTorch Lightning viene con algunos callbacks integrados que se usan regularmente.
Obtenga más información sobre callbacks en PyTorch Lightning [aquí](https://pytorch-lightning.readthedocs.io/en/latest/extensions/callbacks.html).

### <span style="color:#4CC9F0">Devoluciones  de llamada integrados</span> 

En este tutorial, utilizaremos las devoluciones de llamada integradas de parada anticipada `Early Stopping` y punto de control `checkpoint` de modelo. Estos callback se pueden pasar al entrenador (`Trainer`).

### <span style="color:#4CC9F0">Devoluciones de llamada personalizados</span>

Si está familiarizado con la devolución de llamada personalizada de `Keras`, la capacidad de hacer lo mismo en su canalización de PyTorch es solo una cereza del pastel.

Dado que estamos realizando una clasificación de imágenes, la capacidad de visualizar las predicciones del modelo en algunas muestras de imágenes puede resultar útil. Esto en forma de callback puede ayudar a depurar el modelo en una etapa temprana. Así que vamos a implementar un callback personalizado para tal fin.

In [6]:
class ImagePredictionLogger(Callback):
    def __init__(self, val_samples, num_samples=32):
        """
        ImagePredictionLogger es una subclase de la clase Callback de PyTorch Lightning. 
        params:
        val_samples: es una tupla de imágenes y etiquetas. 
        num_samples: es el número de imágenes que se registrarán en el panel de control de W&B.
        """
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
    
    def on_validation_epoch_end(self, trainer, pl_module):
        """
        Este método es llamado cuando finaliza la época de validación. 
        Se necesitan dos argumentos: 
        trainer: entrenador del modelo
        pl_module: módulo del modelo
        Ambios parámetros son pasadoa autompaticamente por el trainer.
        Al usar trainer.logger.experimental podemos usar todas las funciones disponibles por Pesos y sesgos.
        """
        # Trasladar los tensors a CPU
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        # Obtener la predicción del modelo
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        # Pasa la imágenes al logger como  wandb Image
        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            })

        

## <span style="color:blue">Define el modelo: backbone</span> 

Esta clase define el modelo de red neuronal que usaremos para nuestra investigación. Aunque en ocasiones elm modelo hacer parte del sistema que definimos en la siguiente sección, desacoplar el código de esta forma lo hace más agnóstico y fácil de reutilizar.

In [9]:
class Backbone(torch.nn.Module):
    def __init__(self, input_shape, num_classes):
        """
        Modelo neorual de clasificación paar los datos CIFAR10.
        parámetros:
        input_shape: tamaño de los tensores de entrada
        num_classes: número de clases en el problema
        """
        super().__init__()
        
        # log hyperparameters
        #self.save_hyperparameters()
        #self.learning_rate = learning_rate
        
        # bloque convolucional: cuerpo de la red
        """
        CLASS torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
        padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        """
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.conv4 = nn.Conv2d(64, 64, 3, 1)
        """
        CLASS torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, 
        return_indices=False, ceil_mode=False)
        """
        self.pool1 = torch.nn.MaxPool2d(2)
        self.pool2 = torch.nn.MaxPool2d(2)
        
        # Calcula el tamaño de salida del bloque convolucional
        n_sizes = self._get_conv_output(input_shape)
        
        # bloque Lineal: cabeza de la red
        self.fc1 = nn.Linear(n_sizes, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
    
    # métodos auxiliares para calcular el tamaño de salida
    # del bloque convolucion. Se requiere ara poder configurar 
    # totalmente la red. No requrido en Keras
    def _get_conv_output(self, shape):
        """
        devuelve el tamaño del tensor de salida del bloque conv.
        """
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        """
        retorna el tensor de características de la salida el bloque conv
        """
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        return x
    
    # modelo
    def forward(self, x):
        """
        Cálculo del modelo. Corre cada vez que se llama al modelo
        """
        # calcula el bloque convolucional
        x = self._forward_features(x)
        # aplana el tensor de salida del bloque convolucional
        x = x.view(x.size(0), -1) # x.size(0) es el tamaño del lote de datos
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1) # log softmax es más estable que softmax
       
        return x


## <span style="color:blue">LightningModule - Definición del sistema</span> 

`LightningModule` define un sistema y no un modelo. Aquí, un sistema agrupa todo el código de investigación en una sola clase para que sea autónomo. LightningModule organiza su código PyTorch en 5 secciones:

* Bucle de entranamiento (training_step)
* Bucle de validación (validation_step)
* Bucle de prueba (test_step)
* Optimizadores (configure_optimizers)


Por lo tanto, se puede construir un modelo agnóstico del conjunto de datos que se puede compartir fácilmente. Construyamos un sistema para la clasificación Cifar-10.


In [33]:
class LitModel(pl.LightningModule):
    def __init__(self,  backbone, learning_rate=2e-4):
        super().__init__()
        # activa log para almacenar los hiperparámetros
        self.save_hyperparameters()
        # modelo
        self.backbone = backbone
        # rata  de aprendizaje para el optimizador
        self.learning_rate = learning_rate
        # métricas
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()
        
    def training_step(self, batch, batch_idx):
        """
        Lightning automatiza la mayor parte del entrenamiento para nosotros, 
        la época y las iteraciones por lotes, todo lo que necesitamos mantener 
        es la lógica del paso de entrenamiento. El método training_step requiere argumentos 
        batch y batch_idx que el Entrenador pasa automáticamente. 
        """
        x, y = batch
        logits = self.backbone(x)
        loss = F.nll_loss(logits, y) # entropía cruzada: negative log likelihood
        
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.train_acc(preds, y)
        self.log('perdida_entrenamiento', loss, on_step=True, on_epoch=True, logger=True)
        self.log('precision_entrenamiento', acc, on_step=True, on_epoch=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        """
        el ciclo de validación se puede implementar sobrescribiendo este método 
        de LightningModule
        """
        x, y = batch
        logits = self.backbone(x)
        loss = F.nll_loss(logits, y)

        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.valid_acc(preds, y)
        self.log('perdida_validacion', loss, prog_bar=True)
        self.log('precision_validacion', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        """
        Este método similar al ciclo de validación. 
        La única diferencia es que en prueba solo se llama 
        cuando se usa trainer.test(). 
        Las métricas se registran automáticamente por épocas.
        """
        x, y = batch
        logits = self.backbone(x)
        loss = F.nll_loss(logits, y)
        
        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.test_acc(preds, y)
        self.log('perdida_test', loss, prog_bar=True)
        self.log('precision_test', acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        """
        Podemos definir nuestro optimizador y programadores 
        de tasa de aprendizaje usando el método 
        configure_optimizer. 
        Incluso se pueden definir múltiples optimizadores 
        como en el caso de las GAN.
        """
        optimizer = torch.optim.Adam(self.backbone.parameters(), lr=self.learning_rate)
        return optimizer
    
    


## <span style="color:blue">Entrenamiento y Evaluación</span>

Ahora que organizamos nuestra canalización de datos con `DataModule` y modelamos la arquitectura del modelo con `nn.Module` y el ciclo de entrenamiento con `LightningModule`, PyTorch Lightning `Trainer` automatiza todo lo demás por nosotros.

El Entrenador automatiza:

* Iteración de época y lote
* Llamada de `optimizer.step()`, `backward`, `zero_grad()`
* Llamada de `.eval()`, habilitación/deshabilitación de gradientes
* Guardar y cargar pesos
* Registro de pesos y sesgos
* Soporte de entrenamiento multi-GPU
* Soporte de TPU
* soporte de entrenamiento de 16 bits


Primero inicializaremos nuestra canalización de datos. El Entrenador solo necesita un  DataLoader de PyTorch para los datos de entrenamiento/val/prueba. 

Podemos pasar directamente el objeto `dm` que hemos creado al Entrenador. Pero dado que necesitamos algunas muestras para nuestro `ImagePredictionLogger`, llamaremos manualmente a los métodos *prepare_data* y *setup*.

### <span style="color:#4CC9F0">Instancia los datos</span>

In [13]:
# Inicializa la canalización de los datos
dm = CIFAR10DataModule(batch_size=32)
# Para acceder al x_dataloader necesitamos llamar a prepare_data y setup.
dm.prepare_data()
dm.setup()

# Muestras requeridas por el callback personalizado
# ImagePredictionLogger para registrar predicciones de imágenes.  
val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape


Files already downloaded and verified
Files already downloaded and verified


(torch.Size([32, 3, 32, 32]), torch.Size([32]))

Solo necesitamos inicializar el modelo y nuestro logger favorito. Tenga en cuenta que hemos pasado checkpoint_callback por separado. 

### <span style="color:#4CC9F0">Instancia el modelo</span>

In [43]:
# Instancia el modelo
backbone = Backbone(dm.size(), dm.num_classes)
backbone

Backbone(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [44]:
model = LitModel(backbone)

/home/alvaro/anaconda3/envs/ligthning/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:244: UserWarning: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
  rank_zero_warn(


### <span style="color:#4CC9F0">Instancia el trainer</span>

In [50]:
# Inicializa el logger wandb
#wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')

# instancia el trainer
#trainer = pl.Trainer(gpus=4, precisión=16, limit_train_batches=0.5)
max_epochs = 10
acelerator = 'cpu'
gpus = 0
progress_bar_refresh_rate=1

# reproducibilidad
seed_everything(42, workers=True)
# Al escribir la línea anterior es necesario escribir. 
# Caso contrario deterministic=False, que se tiene por defecto
deterministic=True
# Cuántas gpus usar
# logger
# logger=wandb_logger,
logger = True, #TensorBoardLogger

# ModelCheckpoint
checkpoint_callback = ModelCheckpoint(dirpath="../", save_top_k=2, monitor="val_loss")
"""
trainer = pl.Trainer(max_epochs=max_epochs,
                     progress_bar_refresh_rate=progress_bar_refresh_rate, 
                     gpus=gpus, 
                     accelerator=acelerator,                     
                     #logger=logger,
                     callbacks=[EarlyStopping(monitor="val_loss", mode="min"),
                                ImagePredictionLogger(val_samples)],
                     checkpoint_callback=checkpoint_callback,
                     deterministic=deterministic)
"""


Global seed set to 42
/home/alvaro/anaconda3/envs/ligthning/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f23d5694280>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f23d5694280>)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [52]:
trainer = pl.Trainer(accelerator=acelerator, max_epochs= max_epochs, 
                     deterministic=deterministic, gpus=gpus)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### <span style="color:#4CC9F0">Entrena el modelo</span>

In [53]:
# Entrena el modelo
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified



  | Name      | Type     | Params
---------------------------------------
0 | backbone  | Backbone | 952 K 
1 | train_acc | Accuracy | 0     
2 | valid_acc | Accuracy | 0     
3 | test_acc  | Accuracy | 0     
---------------------------------------
952 K     Trainable params
0         Non-trainable params
952 K     Total params
3.809     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

### <span style="color:#4CC9F0">Evalua el modelo</span>

In [ ]:
# Evalua el modelo en el conjunto de prueba retenido
trainer.test()


# Cierra el logger wandb.
#wandb.finish()

In [55]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [57]:
!tensorboard dev upload --logdir \
    'lightning_logs/'

TensorFlow installation not found - running with reduced feature set.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

lightning_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) ^C
Traceback (most recent call last):
  File "/home/alvaro/anaconda3/envs/ligthning/bin/tensorboard", line 10, in <module>
    sys.exit(run_main())
  File "/home/alvaro/anaconda3/envs/ligthning/lib/python3.9/site-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.